# Modelado

In [ ]:
# import libraries
import pandas as pd
from optimus import Optimus
op = Optimus()

In [ ]:
# Read the data
# Data from http://rpubs.com/rhuebner/HRCodebook-13
df = op.read.csv("data/hr-data.csv", header=True)

In [ ]:
# See the data
df.table()

In [ ]:
# Basic data cleaning
from pyspark.sql.functions import when, count, col, isnull


integer_cols = ["MarriedID","MaritalStatusID", "EmpStatusID", "DeptID", "PerfScoreID", "PositionID", "Termd", "ManagerID", 
                "EmpSatisfaction", "SpecialProjectsCount", "DaysLateLast30", "GenderID"]
float_cols = ["PayRate", "EngagementSurvey"]

for col_name in integer_cols:
    df = df.withColumn(col_name, col(col_name).cast('int'))
    
for col_name in float_cols:
    df = df.withColumn(col_name, col(col_name).cast('float'))
    
df = df.dropna(how="all")
df = df.cols.years_between("DOB", date_format="mm/dd/yy",output_cols="Age")

from pyspark.ml.feature import Bucketizer
splits = [10, 20, 30, 40, 50, 60, float("inf")]

bucketizer = Bucketizer(splits=splits, inputCol="Age", outputCol="Age_bucket")
df = bucketizer.transform(df)

In [ ]:
# Número de datos faltantes por columna
from pyspark.sql.functions import when, count, col, isnull

df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).table()

In [ ]:
df = df.dropna(how="any", subset=["ManagerID"])

In [ ]:
# Número de datos faltantes por columna
from pyspark.sql.functions import when, count, col, isnull

df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).table()

In [ ]:
from optimus.ml import feature as fe
df = fe.vector_assembler(df,input_cols=['MaritalStatusID', 'GenderID', 'DeptID', 'PayRate', "ManagerID", 
                                   'EmpSatisfaction', 'Age_bucket'], output_col="features")

In [ ]:
from pyspark.ml.feature import ChiSqSelector

selector = ChiSqSelector(numTopFeatures=5, featuresCol="features",
                         outputCol="selected_features", labelCol="Termd")

result = selector.fit(df).transform(df)

In [ ]:
result.table()

In [ ]:
# Train model.  This also runs the indexers.
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="Termd", featuresCol="selected_features", maxIter=10)
model = gbt.fit(result)

In [ ]:
# Make predictions.
predictions = model.transform(result)

In [ ]:
# Select example rows to display.
predictions.select("prediction", "Termd", "selected_features").table()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="Termd", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

In [ ]:
accuracy

## Mejorando el modelo

In [ ]:
# División de datos
(training_data, test_data) = result.randomSplit([0.7, 0.3])

In [ ]:
# Train model.  This also runs the indexers.
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="Termd", featuresCol="selected_features", maxIter=10)
model = gbt.fit(training_data)

# Make predictions.
predictions = model.transform(test_data)

In [ ]:
# Select example rows to display.
predictions.select("prediction", "Termd", "selected_features").table()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="Termd", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

In [ ]:
model.featureImportances